In [ ]:
import math
import sqlite3
import hashlib

class Funcionario:
    """Classe para representar um funcionário."""
    def _init_(self, nome, cargo, salario):
        self.nome = nome
        self.cargo = cargo
        self.salario = salario

class Produto:
    """Classe para representar um produto."""
    def _init_(self, nome, preco):
        self.nome = nome
        self.preco = preco

def read_double(prompt):
    """Lê um número de ponto flutuante do usuário com tratamento de erros."""
    while True:
        try:
            value = float(input(prompt))
            if value < 0:
                print("Por favor, insira um número positivo.")
                continue
            return value
        except ValueError:
            print("Entrada inválida. Por favor, insira um número válido.")

def hash_password(password):
    """Hash a senha usando SHA-256."""
    return hashlib.sha256(password.encode()).hexdigest()

def init_db(conn=None):
    """Inicializa o banco de dados e cria as tabelas necessárias."""
    if conn is None:
        conn = sqlite3.connect('simulador_financeiro.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS usuarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE,
        password TEXT
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS funcionarios (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        cargo TEXT,
        salario REAL
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS produtos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nome TEXT,
        preco REAL
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS vendas (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        produto_id INTEGER,
        vendas INTEGER,
        FOREIGN KEY (produto_id) REFERENCES produtos (id)
    )''')
    cursor.execute('''CREATE TABLE IF NOT EXISTS historico_simulacoes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ganho_planejado REAL,
        custo_material REAL,
        gasto_fixo_mensal REAL,
        lucro_necessario REAL,
        investimento_reservado REAL,
        emergencia_reservada REAL,
        lucro_liquido REAL
    )''')
    conn.commit()
    return conn



def create_user(conn, username, password):
    """Cria um novo usuário no banco de dados."""
    cursor = conn.cursor()
    cursor.execute('INSERT INTO usuarios (username, password) VALUES (?, ?)',
                   (username, hash_password(password)))
    conn.commit()

def authenticate_user(conn, username, password):
    """Verifica se as credenciais do usuário estão corretas."""
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM usuarios WHERE username = ? AND password = ?',
                   (username, hash_password(password)))
    return cursor.fetchone() is not None

def inserir_funcionario(conn, funcionario):
    """Insere um funcionário no banco de dados."""
    cursor = conn.cursor()
    cursor.execute('INSERT INTO funcionarios (nome, cargo, salario) VALUES (?, ?, ?)',
                   (funcionario.nome, funcionario.cargo, funcionario.salario))
    conn.commit()

def inserir_produto(conn, produto):
    """Insere um produto no banco de dados."""
    cursor = conn.cursor()
    cursor.execute('INSERT INTO produtos (nome, preco) VALUES (?, ?)',
                   (produto.nome, produto.preco))
    conn.commit()
    return cursor.lastrowid  # Retorna o ID do produto inserido

def inserir_venda(conn, produto_id, vendas):
    """Insere as vendas no banco de dados."""
    cursor = conn.cursor()
    cursor.execute('INSERT INTO vendas (produto_id, vendas) VALUES (?, ?)',
                   (produto_id, vendas))
    conn.commit()

def inserir_historico(conn, ganho_planejado, custo_material, gasto_fixo_mensal, lucro_necessario, investimento_reservado, emergencia_reservada, lucro_liquido):
    """Insere os dados da simulação no histórico."""
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO historico_simulacoes (ganho_planejado, custo_material, gasto_fixo_mensal, lucro_necessario, investimento_reservado, emergencia_reservada, lucro_liquido)
    VALUES (?, ?, ?, ?, ?, ?, ?)''', (ganho_planejado, custo_material, gasto_fixo_mensal, lucro_necessario, investimento_reservado, emergencia_reservada, lucro_liquido))
    conn.commit()

def register(conn):
    """Função para registrar um novo usuário."""
    username = input("Escolha um nome de usuário: ")
    password = input("Escolha uma senha: ")
    create_user(conn, username, password)
    print("Usuário criado com sucesso!")

def login(conn):
    """Função para fazer login de um usuário."""
    username = input("Nome de usuário: ")
    password = input("Senha: ")
    if authenticate_user(conn, username, password):
        print("Login bem-sucedido!")
        return True
    else:
        print("Nome de usuário ou senha incorretos.")
        return False

def ver_historico(conn):
    """Exibe o histórico das simulações."""
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM historico_simulacoes ORDER BY id DESC LIMIT 1')
    resultado = cursor.fetchone()
    if resultado:
        print("\n--- Última Simulação ---")
        print(f"Ganho Planejado: R$ {resultado[1]:.2f}")
        print(f"Custo de Materiais: R$ {resultado[2]:.2f}")
        print(f"Gasto Fixo Mensal: R$ {resultado[3]:.2f}")
        print(f"Lucro Necessário: R$ {resultado[4]:.2f}")
        print(f"Investimento Reservado: R$ {resultado[5]:.2f}")
        print(f"Emergência Reservada: R$ {resultado[6]:.2f}")
        print(f"Lucro Líquido: R$ {resultado[7]:.2f}")
    else:
        print("Nenhuma simulação registrada.")

def analisar_lucro(custo_material, numero_funcionarios, vendas_totais):
    """Análise preditiva de lucro simples baseada em custo e funcionários."""
    if vendas_totais > 0:
        lucro_medio = (vendas_totais - (custo_material + numero_funcionarios * 1000)) / vendas_totais  # Exemplo de custo fixo
        print(f"Lucro médio por venda: R$ {lucro_medio:.2f}")
    else:
        print("Sem vendas registradas para análise.")

def recomendar_precos(produtos):
    """Recomenda preços com base em um aumento simples."""
    print("\n--- Recomendações de Preços ---")
    for produto in produtos:
        preco_recomendado = produto.preco * 1.1  # Aumenta o preço em 10%
        print(f"Recomenda-se aumentar o preço de {produto.nome} para R$ {preco_recomendado:.2f}")

def analisar_risco(ganho_planejado, custo_material, gasto_fixo_mensal):
    """Análise de risco simples."""
    risco = (custo_material + gasto_fixo_mensal) / ganho_planejado
    if risco > 1:
        print("Alerta: O risco financeiro é elevado! Cuidado com os gastos.")
    else:
        print("Risco financeiro dentro dos limites aceitáveis.")

def main():
    """Função principal que executa o simulador financeiro."""
    conn = init_db()  # Inicializa o banco de dados
    print("\033[1;34mBem-vindo ao Simulador Financeiro!\033[0m")  # Azul

    while True:
        action = input("Digite '1' para entrar, '2' para criar uma conta ou '3' para ver o histórico: ").strip()
        if action == '1':
            if login(conn):
                ver_historico(conn)  # Chama o histórico após login bem-sucedido
                break  # Sai do loop se o login for bem-sucedido
        elif action == '2':
            register(conn)
        elif action == '3':
            ver_historico(conn)
        else:
            print("Ação inválida. Tente novamente.")

    print("Este programa ajudará você a atingir suas metas financeiras.")

    # Perguntas sobre ganho e custos
    ganho_planejado = read_double("Qual é a sua renda planejada? R$ ")
    custo_material = read_double("Qual é o custo total de materiais? R$ ")
    gasto_fixo_mensal = read_double("Qual é o gasto fixo mensal (aluguel, energia, água, etc.)? R$ ")
    percentual_investimento = read_double("Qual a porcentagem do lucro que você deseja investir? (em %) ")
    percentual_emergencia = read_double("Qual a porcentagem do lucro que você deseja reservar para emergências? (em %) ")

    # Perguntas sobre funcionários
    numero_funcionarios = int(read_double("Quantas pessoas trabalham para você? "))
    funcionarios = []

    for i in range(numero_funcionarios):
        print(f"\nInforme os dados do funcionário {i + 1}:")
        nome = input("Nome: ")
        cargo = input("Cargo: ")
        salario = read_double("Salário: R$ ")
        funcionario = Funcionario(nome, cargo, salario)
        funcionarios.append(funcionario)
        inserir_funcionario(conn, funcionario)  # Insere no banco de dados

    # Perguntas sobre produtos
    numero_produtos = int(read_double("Quantos produtos você vende? "))
    produtos = []

    for i in range(numero_produtos):
        print(f"\nInforme os dados do produto {i + 1}:")
        nome = input("Nome: ")
        preco = read_double("Preço: R$ ")
        produto = Produto(nome, preco)
        produtos.append(produto)
        produto_id = inserir_produto(conn, produto)  # Insere no banco de dados

        # Pergunta pela venda total
        vendas = read_double(f"Quantas unidades de {produto.nome} você vendeu no último mês? ")
        inserir_venda(conn, produto_id, vendas)  # Insere a venda no banco de dados

    # Cálculos financeiros
    custo_total_funcionarios = sum(funcionario.salario for funcionario in funcionarios)

    # Cálculo do lucro necessário
    lucro_necessario = ganho_planejado - (custo_material + custo_total_funcionarios + gasto_fixo_mensal)

    if lucro_necessario < 0:
        print("\033[1;31mO lucro necessário é negativo. Você precisa aumentar sua receita ou reduzir custos.\033[0m")  # Vermelho
        return

    # Cálculo da receita total
    receita_total = lucro_necessario + custo_material + custo_total_funcionarios + gasto_fixo_mensal

    # Cálculo de vendas necessárias
    vendas_necessarias = math.ceil(lucro_necessario / produtos[0].preco)  # arredondando para cima
    numero_vendas = math.ceil(receita_total / produtos[0].preco)  # arredondando para cima

    # Cálculo de reservas
    investimento_reservado = (lucro_necessario * percentual_investimento) / 100
    emergencia_reservada = (lucro_necessario * percentual_emergencia) / 100
    lucro_liquido = lucro_necessario - (investimento_reservado + emergencia_reservada)

    # Cálculo de vendas diárias
    dias_no_mes = 30
    vendas_diarias = math.ceil(numero_vendas / dias_no_mes)

    print("\n\033[1;33mAtenção:\033[0m gastos podem ser variáveis de acordo com o mercado, por isso é importante sempre ter um dinheiro de emergência.")

    # Resultados
    print("\n--- Resultados Finais ---")
    print(f"Receita Total: R$ {receita_total:.2f}")
    print(f"Custo de Materiais: R$ {custo_material:.2f}")
    print(f"Custo Total de Funcionários: R$ {custo_total_funcionarios:.2f}")
    print(f"Custo Total de Gastos Fixos: R$ {gasto_fixo_mensal:.2f}")
    print(f"Lucro Necessário: R$ {lucro_necessario:.2f}")
    print(f"Investimento Reservado: R$ {investimento_reservado:.2f}")
    print(f"Emergência Reservada: R$ {emergencia_reservada:.2f}")
    print(f"Lucro Líquido após reservas: R$ {lucro_liquido:.2f}")

    print(f"Número estimado de vendas total: {numero_vendas}")
    print(f"Número estimado de vendas diárias: {vendas_diarias}")

    for produto in produtos:
        vendas_produto_diarias = math.ceil(vendas_diarias / numero_produtos)  # Distribuindo igualmente
        print(f"Você precisaria vender {vendas_produto_diarias} unidades de {produto.nome} por dia.")

    # Inserir os dados da simulação no histórico
    inserir_historico(conn, ganho_planejado, custo_material, gasto_fixo_mensal, lucro_necessario, investimento_reservado, emergencia_reservada, lucro_liquido)

    # Análises e recomendações
    vendas_totais = sum([read_double(f"Quantas unidades de {produto.nome} você vendeu no último mês? ") for produto in produtos])
    analisar_lucro(custo_material, numero_funcionarios, vendas_totais)
    recomendar_precos(produtos)
    analisar_risco(ganho_planejado, custo_material, gasto_fixo_mensal)

    print("\nObrigado por usar o Simulador Financeiro! Boa sorte em suas empreitadas.")
    conn.close()  # Fecha a conexão com o banco de dados

    if _name_ == "_main_":
      main()